In [1]:
print('Hello LangChain')

Hello LangChain


In [2]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [3]:
# prompt
prompt = ChatPromptTemplate.from_messages(
    [ ("system", "당신은 개발자입니다.") , 
     ("user", "{input}") ]
)
print(prompt)

prompt_text = prompt.format(input="자바는 무엇인가요? 자세하게 설명해주세요")

input_variables=['input'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 개발자입니다.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})]


In [4]:
# Groq API를 사용하는 ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)
print(llm)

client=<openai.resources.chat.completions.completions.Completions object at 0x000001EDEF094B60> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001EDEF7CF8C0> root_client=<openai.OpenAI object at 0x000001EDEF19A6F0> root_async_client=<openai.AsyncOpenAI object at 0x000001EDEF6576B0> model_name='meta-llama/llama-4-scout-17b-16e-instruct' temperature=0.7 model_kwargs={} openai_api_key=SecretStr('**********') openai_api_base='https://api.groq.com/openai/v1'


### LCEL
* Prompt + LLM을 Chain으로 연결하기

In [5]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("" \
"You are an expert in AI Expert." \
"Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.")
prompt

PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='You are an expert in AI Expert.Answer the question. <Question>: {input}에 대해 쉽게 설명해주세요.')

In [6]:
# chain 연결 (LCEL)
chain = prompt | llm
print(type(chain))

<class 'langchain_core.runnables.base.RunnableSequence'>


### LCEL
* Prompt + LLM + outputparser를 Chain으로 연결하기

In [7]:
from langchain_core.output_parsers import StrOutputParser

# chain 연결 (LCEL) prompt + llm + outputparser
output_parser = StrOutputParser()

chain2 = prompt | llm | output_parser
print(type(chain2))

<class 'langchain_core.runnables.base.RunnableSequence'>


In [8]:
# chain 호출
try:
    result = chain.invoke({"input": "인공지능 모델의 학습 원리"})
    print(type(result))
    print(result.content)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'langchain_core.messages.ai.AIMessage'>
인공지능 모델의 학습 원리는 사람의 뇌가 학습하는 원리와 유사합니다. 컴퓨터에大量的한 데이터를 제공하고, 데이터를 분석하고 패턴을 학습하도록 하는 것이죠.

예를 들어, 고양이와 강아지의 사진을 분류하는 모델을 만든다고 가정해 봅시다. 이 모델에게 수백만 장의 고양이와 강아지 사진을 보여주고, 각각의 사진이 고양이인지 강아지인지를 알려줍니다.

모델은 처음에는 고양이와 강아지의 특징을 잘 모릅니다. 하지만 사진을 계속 보여주고, 고양이인지 강아지인지를 반복적으로 알려주면 모델은 점차 고양이와 강아지의 특징을 학습하기 시작합니다.

이를테면, 고양이는 귀가 뾰족하고, 눈이 크며, 털이 보드라운 특징이 있다는 것을 학습할 수 있습니다. 강아지는 귀가 쳐져 있고, 꼬리가 길며, 털이 거친 특징이 있다는 것을 학습할 수 있습니다.

이렇게 모델은 데이터를 학습하면서, 고양이와 강아지를 구분할 수 있는 기준을 스스로 만들어냅니다. 이 기준을 바탕으로 새로운 사진을 보여주면, 모델은 고양이인지 강아지인지를 정확하게 분류할 수 있습니다.

이러한 학습 과정을 통해 인공지능 모델은 데이터를 분석하고, 패턴을 학습하며, 예측과 분류를 수행할 수 있습니다.

보다 기술적으로 설명하면, 인공지능 모델의 학습 원리는 다음과 같습니다:

1. **데이터 수집**: 인공지능 모델을 학습시키기 위해大量的한 데이터를 수집합니다.
2. **데이터 전처리**: 수집한 데이터를 전처리하여 모델이 학습할 수 있는 형태로 변환합니다.
3. **모델 정의**: 학습할 모델을 정의하고, 모델의 구조를 결정합니다.
4. **학습**: 모델에 데이터를 제공하고, 모델이 데이터를 학습하도록 합니다.
5. **평가**: 모델의 성능을 평가하고, 필요에 따라 모델을 수정하거나 재학습합니다.

이러한 과정을 통해 인공지능 모델은 데이터를 학습하고, 예측과 분류를 수행할 수 있습니다.


In [9]:
# chain 호출
try:
    result = chain2.invoke({"input": "LangChain의 Products(제품)는 어떤 것들이 있나요? 예를 들어 LangSmith, LangServe같은 products가 있어."})
    print(type(result))
    print(result)
except Exception as e:
    print(f"오류 발생: {e}")

<class 'str'>
LangChain은 다양한 제품과 도구를 제공하여 개발자들이 언어 모델을 쉽게 통합하고 활용할 수 있도록 지원합니다. 주요 제품은 다음과 같습니다:

1. **LangSmith**: LangSmith는 LangChain에서 제공하는 통합 개발 환경(IDE)입니다. LangSmith를 사용하면 개발자들이 언어 모델을 쉽게 개발, 테스트 및 배포할 수 있습니다. LangSmith는 모델 개발, 평가, 배포를 위한 다양한 도구와 기능을 제공합니다.

2. **LangServe**: LangServe는 LangChain에서 제공하는 API 서비스입니다. LangServe를 사용하면 개발자들이 언어 모델을 쉽게 API로 배포하고 관리할 수 있습니다. LangServe는 모델의 배포, 관리, 모니터링을 위한 다양한 기능을 제공합니다.

3. **LangChain 자체**: LangChain은 다양한 언어 모델을 통합하고 활용할 수 있는 프레임워크입니다. LangChain은 개발자들이 언어 모델을 쉽게 통합하고 활용할 수 있도록 지원합니다.

LangChain의 제품들은 개발자들이 언어 모델을 쉽게 개발, 배포 및 관리할 수 있도록 지원합니다. 이러한 제품들은 자연어 처리, 대화형 AI, 언어 번역 등 다양한 분야에서 활용될 수 있습니다.

쉽게 말해, LangChain은 언어 모델을 활용하기 위한 플랫폼을 제공하고, LangSmith와 LangServe는 그 플랫폼을 활용하여 모델을 개발하고 배포하는 데 도움을 주는 도구들입니다.


### Runnable의 stream() 함수 호출

In [10]:
# 스트리밍 출력을 위한 요청
try:
    answer = chain2.stream({"input": "인공지능 모델의 학습 원리를 자세하게 설명해주세요."})
    # 스트리밍 출력
    print(answer)

    for token in answer:
        # 스트림에서 받은 데이터의 내용을 출력합니다. 줄바꿈 없이 이어서 출력하고, 버퍼를 즉시 비웁니다.
        print(token, end="", flush=True)
except Exception as e:
    print(f"오류 발생: {e}")

<generator object RunnableSequence.stream at 0x000001EDEF623970>
인공지능 모델의 학습 원리는 사람의 뇌가 학습하는 방식과 유사합니다. 컴퓨터가 데이터를 통해 스스로 학습하고, 패턴을 발견하며, 미래의 새로운 데이터에 대해 예측하거나 결정을 내릴 수 있도록 하는 것입니다.

### 1. 데이터 수집
인공지능 모델 학습의 첫 단계는 데이터를 수집하는 것입니다. 이 데이터는 과거의 경험에서 얻어진 정보로, 모델이 학습하고 예측해야 할 대상과 관련된 것입니다. 예를 들어, 이미지 인식 모델을 만든다면, 수많은 이미지 데이터를 수집해야 합니다.

### 2. 데이터 전처리
수집된 데이터는 대부분이 모델에 바로 사용될 수 있는 상태가 아닙니다. 따라서 데이터를 정제하고, 필요한 경우 데이터를 변환하는 과정이 필요합니다. 예를 들어, 이미지의 크기를 조정하거나, 텍스트 데이터의 경우 특수 문자나 불필요한 단어를 제거하는 작업 등이 이에 해당합니다.

### 3. 모델 선택
적합한 인공지능 모델을 선택해야 합니다. 모델의 종류는 목적에 따라 다양합니다. 예를 들어, 이미지 분류에는 합성곱 신경망(CNN), 자연어 처리에는 순환 신경망(RNN) 또는 트랜스포머 모델을 사용할 수 있습니다.

### 4. 학습
선택한 모델에 데이터를 입력하여 모델을 학습시킵니다. 학습 과정에서는 모델이 입력 데이터로부터 패턴을 학습하고, 그 패턴을 통해 예측을 할 수 있도록 합니다. 이 과정은 '손실 함수(loss function)'를 통해 모델의 예측 오류를 계산하고, 오류를 최소화하는 방향으로 모델의 가중치를 조정하는 방식으로 진행됩니다.

### 5. 최적화
모델의 성능을 최적화하기 위해 '경사 하강법(gradient descent)'과 같은 최적화 알고리즘을 사용합니다. 이 알고리즘은 손실 함수의 기울기를 계산하여, 손실이 감소하는 방향으로 모델의 파라미터를 조정합니다.

### 6. 평가
학습된 모델의 성능을 평가합니다. 이를 위해 학

### Multi Chain
* 첫 번째 Chain의 출력이, 두 번째 Chain의 입력이 된다.
* 두 개의 Chain과 Prompt + OutputParser를 LCEL로 연결하기